# Binding Theory #

principle A: an **anaphor** must be bound within its local domain D

principle B: a **pronoun** must be free within its local domain D

principle C: an **r-expression** must be free

## purpose: principle A ##

Binding relation:
1.   check c-command
2.   check coindexation



In [0]:
import nltk
import re
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from nltk.tree import *
from nltk import Nonterminal, nonterminals, Production, CFG
!pip install git+https://github.com/emilmont/pyStatParser
from stat_parser import Parser

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
  Cloning https://github.com/emilmont/pyStatParser to /tmp/pip-req-build-x_xqy7ze
  Running command git clone -q https://github.com/emilmont/pyStatParser /tmp/pip-req-build-x_xqy7ze
  Created wheel for pyStatParser: filename=pyStatParser-0.0.1-cp36-none-any.whl size=720546 sha256=e8c7361176d7d25e7a10b37c2c62d0fb23e5383823839dc8131938e6d2c57793
  Stored in directory: /tmp/pip-ephem-wheel-cache-tdriaeqr/wheels/bd/8b/c5/49a6cdbdd3b046c25475a132c1b7a0be3cdd2b3562d01becdd
Successfully built pyStatParser


In [0]:
from stat_parser import Parser
parser = Parser()
example = "How can the net amount of entropy of the universe be massively decreased?"
tree = parser.parse(example)
print (tree)
print (tree.treepositions())

(SBARQ
  (WHADVP (WRB how))
  (SQ
    (MD can)
    (NP
      (NP (DT the) (JJ net) (NN amount))
      (PP
        (IN of)
        (NP
          (NP (NNS entropy))
          (PP (IN of) (NP (DT the) (NN universe))))))
    (VP (VB be) (ADJP (RB massively) (VBN decreased))))
  (. ?))
[(), (0,), (0, 0), (0, 0, 0), (1,), (1, 0), (1, 0, 0), (1, 1), (1, 1, 0), (1, 1, 0, 0), (1, 1, 0, 0, 0), (1, 1, 0, 1), (1, 1, 0, 1, 0), (1, 1, 0, 2), (1, 1, 0, 2, 0), (1, 1, 1), (1, 1, 1, 0), (1, 1, 1, 0, 0), (1, 1, 1, 1), (1, 1, 1, 1, 0), (1, 1, 1, 1, 0, 0), (1, 1, 1, 1, 0, 0, 0), (1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 0), (1, 1, 1, 1, 1, 0, 0), (1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 0), (1, 1, 1, 1, 1, 1, 0, 0), (1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 0), (1, 2), (1, 2, 0), (1, 2, 0, 0), (1, 2, 1), (1, 2, 1, 0), (1, 2, 1, 0, 0), (1, 2, 1, 1), (1, 2, 1, 1, 0), (2,), (2, 0)]


In [0]:
def dominate(tree, node1, node2):
  for i in tree.treepositions():
    if tree[i] == node1:
      i_node1 = i[:-1]
    if tree[i] == node2:
      i_node2 = i[:-1]

  if i_node2 in i_node1:
    return True
  else:
    return False

def parent_of(tree, node1, node2):
  for i in tree.treepositions():
   if tree[i] == node1:
      i_node1 = i[:-1]
   if tree[i] == node2:
      i_node2 = i[:-1]
  
  if i_node1[:] == i_node2[:-1]:
    return True
  else:
    return False

def sibling_with(tree, node1, node2):
  for i in tree.treepositions():
   if tree[i] == node1:
      i_node1 = i[:-1]
   if tree[i] == node2:
      i_node2 = i[:-1]

  if i_node1[:-1] == i_node2[:-1]:
    return True
  else:
    return False

In [0]:
def c_command(tree, node1, node2):
  for i in tree.treepositions():
    if tree[i] == node1:
      i_node1 = i[:-1]
    if tree[i] == node2:
      i_node2 = i[:-1]
      
  diff = (len(i_node2) - len(i_node1))
  if sibling_with(tree, node1, node2) and i_node1 != i_node2:
    return True
  elif diff > 0 and i_node1[-1] != i_node2[-(diff)]:
    return True
  else:
    return False               

In [0]:
tree = parser.parse("He looked at himself in the mirror")
c_command(tree, "He", "himself")

UnboundLocalError: ignored

In [0]:
def binding_a(text):
  split_text = text.lower().split()
  tagged = pos_tag(split_text)
  
  pronoun = ["PRP", "PRP$"]
  noun = ["NN", "NNS", "NNP", "NNPS", "FW"]
  anaphora = ["himself", "herself", "yourself", "myself", "themselves", "ourself", "ourselves", "itself"]

  sent_noun = []
  sent_pron = []

  for i in range(len(tagged)):
    if tagged[i][1] in noun:
      sent_noun.append(tagged[i][0])
    if tagged[i][1] in pronoun:
      sent_pron.append(tagged[i][0])

  # Ask users for the reference of the pronouns
  reference = []
  for x in sent_pron:
    ref = input("What do you mean by '"+x+"'?: ").lower()
    reference.append((x,ref))

  # Make tree and find out if the anaphora in tree is properly working
  tree = parser.parse(text)

  for i in range(len(reference)):
    for j in range(len(reference)):
      for k in range(len(sent_noun)):
        
        # case1: more than one pronouns are in binding relation
        if reference[i][1] == reference[j][1] and c_command(tree, reference[i][0], reference[j][0]):
          if reference[j][0] not in anaphora:
            print("Wrong")
        
        # case2: noun and pronouns are in binding relation
        elif reference[i][1] == sent_noun[j] and c_command(tree, reference[i][0], sent_noun[j]):
          if reference[j][0] not in anaphora:
            print("Wrong")
        
        # case3: noun and pronouns are not in binding relation but using anaphora
        elif reference[i][1] != sent_noun[j] or c_command(tree, reference[i][0], sent_noun[j]):
          if reference[j][0] in anaphora:
            print("Wrong")

        else:
          print("Good")
  

In [0]:
text = input("Give me a sentence: ")
binding_a(text)

Give me a sentence: John saw himself
What do you mean by 'himself'?: 
Wrong
